ICI ca test de recup des infos sur nba.com

import all the libraries

In [16]:
from selenium import webdriver #/!\ version :4.5.0
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options


from bs4 import BeautifulSoup

import pandas as pd

import time

Constants

In [17]:
#create a constant for the url
NBA_PLAYERS_START_URL = 'https://www.nba.com/stats/alltime-leaders'

# path to the file containing the chrome driver
CHROME_DRIVER_PATH = 'C:/chromedriver.exe'

Useful functions for browser

In [18]:
# Starting the browser.
def startBrowser():
    s=Service(CHROME_DRIVER_PATH)
    options = webdriver.ChromeOptions()
    options = Options()
    options.add_argument("--headless")
    return  webdriver.Chrome(service=s, options=options)

# Checking if the browser is closed.
def isBrowserClosed(browser):
    isbrowserClosed = False
    try:
        webdriver.title
    except:
        isbrowserClosed = True
    return isbrowserClosed
        

Get stats from all players

In [23]:
# The above code is getting the data from the URL and returning the dataframe.
def getDataFromURL(URL):
    browser = startBrowser()
    #open the url
    browser.get(URL)

    # accept cookies
    browser.find_element(By.CSS_SELECTOR, "button#onetrust-accept-btn-handler").click()
    time.sleep(10)

    df = pd.DataFrame()

    while browser.find_element(By.CSS_SELECTOR, "button[title^='Next Page Button']").is_enabled():
        html = browser.page_source
        data, headers = getDataFromHTML(html)

        df = pd.concat([df, pd.DataFrame(data, columns=headers)], ignore_index=True)

        print(".", end = '')

        while True:
            try:
                browser.find_element(By.CSS_SELECTOR, "button[title^='Next Page Button']").click()
                break
            except:
                if(isBrowserClosed(browser)):
                    return df
                else:
                    print('not yet clickable')
                    continue

        html = browser.page_source
        data, headers = getDataFromHTML(html)

        df = pd.concat([df, pd.DataFrame(data, columns=headers)], ignore_index=True)

        print(".", end = '')

    #close the browser
    browser.quit()
    #return the dataframe
    return df



#Parsing the html and returning the data and headers.
def getDataFromHTML(html):
    #parse the html
    soup = BeautifulSoup(html, 'html.parser')
    #find the table with class Crom_table__p1iZz
    table = soup.find('table', attrs = {'class' : 'Crom_table__p1iZz'})
    #get the table headers
    headers = [header.text for header in table.findAll('th', attrs = {'hidden': None})]
    #get the table rows
    rows = table.find_all('tr')
    #get the table data
    data = [[td.text for td in rows[i].find_all('td')] for i in range(len(rows))]
    data = [row for row in data if row != []]#they is an empty at the start idk why but yes

    return data, headers


Save stats in a csv file

In [24]:
print("start of the scrapping")
df = getDataFromURL(NBA_PLAYERS_START_URL)
print("\nend of the scrapping\n\n")



print("saving the dataframe to a csv file...")
df.to_csv('nbaPlayersData.csv', index = False)
print("done")

start of the scrapping
..............................................................................................................................................................................................
end of the scrapping


saving the dataframe to a csv file...
done
